In [51]:
!pip install -q -U google-generativeai


I0000 00:00:1731792992.612643 7016086 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [62]:
import google.generativeai as genai
from IPython.display import Markdown
from dotenv import load_dotenv
import os
from tenacity import retry
import instructor
import google.generativeai as genai
from pydantic import BaseModel
load_dotenv()

True

In [53]:
gemini_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=gemini_key)


In [54]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2212k      0 --:--:-- --:--:-- --:--:-- 2224k


I0000 00:00:1731792996.771594 7016086 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [55]:
sample_file = genai.upload_file(path="PumpkinPie2-2.jpg",
                            display_name="Jetpack drawing")


sample_file_uri = sample_file.uri
print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

I0000 00:00:1731793001.630281 7016086 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/devv7xdkdrga


In [56]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/devv7xdkdrga


In [76]:
from tenacity import stop_after_attempt


model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
@retry(stop=stop_after_attempt(7))
def generate_image(prompt):
    response = model.generate_content([sample_file, prompt])
    return response
prompt =  "What is the name of the product in the image, do not give a whole sentence and do not describe the image. Also predict the weight of the item in the image in grams "
response = generate_image(prompt)
Markdown(">" + response.text)

>Pumpkin pie

150g (Estimate for a single slice, the whole pie could be around 1000-1200g)

In [77]:
class Description(BaseModel):
    name: str
    weight: int
    

In [ ]:
content = [
    "What is the name of the product in the image, do not give a whole sentence and do not describe the image. Also predict the weight of the item in the image in grams ",
    sample_file,  
]

In [ ]:
client = instructor.from_gemini(
    client=genai.GenerativeModel(
        model_name="models/gemini-1.5-flash-latest",
    ),
    mode=instructor.Mode.GEMINI_JSON,  
)

In [ ]:
resp = client.create(
    response_model=Description,
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
)